<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/crew_scheduling_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sets up flight scheduling problem from set of feasible flight sequence to flights
# Cost of each flight sequence and mapping of sequence to flights are given.
# Maximum number of available crew is given

In [ ]:
pip install pyomo  #Installs the modeling language called pyomo

In [ ]:
# The GLPK (GNU Linear Programming Kit) package is intended for solving large-scale linear programming (LP), 
# mixed integer programming (MIP), and other related problems. It is a set of routines written in ANSI C and 
# organized in the form of a callable library.
!apt-get install -y -qq glpk-utils  #Installs the optimization engine called glpk.


In [ ]:
from pyomo.environ import *

In [ ]:

flights = ['SFO-LAX','SFO-DEN', 'SFO-SEA', 'LAX-ORD', 'LAX-SFO', 'ORD-DEN', \
           'ORD-SEA', 'DEN-SFO', 'DEN-ORD', 'SEA-SFO', 'SEA-LAX' ]    # This is the set of flights

feasible_sequence = list(range(1,13) )                                # This is the sequences (range 1 to 12 converted to a list)

cost_sequence = [2, 3, 4, 6, 7, 5, 7, 8, 9, 9, 8, 9]                  # cost of flight sequence

 # this creates a list with flights and sequence

is_flight_in_sequence = [('SFO-LAX', 1), ('SFO-LAX', 4), \
                        ('SFO-LAX', 7), ('SFO-LAX', 10), \
                         ('SFO-DEN', 2), ('SFO-DEN', 5), ('SFO-DEN', 8), ('SFO-DEN', 11),  \
                          ('SFO-SEA', 3), ('SFO-SEA', 6), ('SFO-SEA', 9), ('SFO-SEA', 12), \
                         ('LAX-ORD', 4), ('LAX-ORD', 7),  ('LAX-ORD', 9), ('LAX-ORD', 10), ('LAX-ORD', 12), \
                         ('LAX-SFO', 1), ('LAX-SFO', 6),  ('LAX-SFO', 10), ('LAX-SFO', 11), \
                         ('ORD-DEN', 4),  ('ORD-DEN', 5), ('ORD-DEN', 9), \
                          ('ORD-SEA', 7), ('ORD-SEA', 8), \
                         ('ORD-SEA', 10), ('ORD-SEA', 11), ('ORD-SEA', 12), \
                         ('DEN-SFO', 2),  ('DEN-SFO', 4), \
                         ('DEN-SFO', 5),  ('DEN-SFO', 9), \
                          ('DEN-ORD', 5), ('DEN-ORD', 8), \
                          ('DEN-ORD', 11), \
                         ('SEA-SFO', 3), ('SEA-SFO', 7), ('SEA-SFO', 8), \
                         ('SEA-SFO', 12), \
                         ('SEA-LAX', 6), \
                         ('SEA-LAX', 9), ('SEA-LAX', 10), ('SEA-LAX', 11), ('SEA-LAX', 12)] 

max_crew = 4                                                          # only 3 crew available


In [ ]:

constraints = {'supply_constraint', 'demand_constraint'}           # The  set of constraints

model = ConcreteModel(name = "(Model2)")                            # Same as previous
model.x = Var( feasible_sequence, within= Binary )       # Decision variables are the flows from - to
model.value = Objective(                                            # Objective
expr = sum( cost_sequence[i-1]*model.x[i] for i in feasible_sequence), sense = minimize )  # Minimize total crew cost

# This defines a rule called demand must be met. 

def demand_must_be_met_rule(m,c):
    return sum(m.x[is_flight_in_sequence[j][1]] for j in range(len(is_flight_in_sequence)) if c == is_flight_in_sequence[j][0]) >= 1    
    # sums whether flight sequences covering flight c add up to greater than one

# This defines in our model the constraints! Note that we simply pass the set of constraints and the rule. It does the rest.
# model is by default when we call (recall model can be renamed as you like )
model.demand_constraint = Constraint(flights, rule = demand_must_be_met_rule)   # applies to each flight

model.supply_constraint = Constraint(expr = sum (model.x[i] for i in feasible_sequence) <= max_crew)  # Max crew size

opt = SolverFactory('glpk')           # same as before

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)   # same as before
results = opt.solve(model, tee= True)                 # same as before

In [ ]:
model.pprint()